**性格によって取る行動が変わるNPCを強化学習によって作る**

In [44]:
#@title import

import numpy as np  # 数値計算
import random  # ランダムな選択
import pickle  # モデルの保存と読み込み

In [45]:
#@title キャラクタークラス

class Character:
    def __init__(self, name, personality, max_hp=100):
        """
        キャラクターの初期化
        :param name: 名前
        :param personality: 性格
        :param hp: HP
        :param max_hp: 最大HP(100)
        """
        self.name = name
        self.personality = personality
        self.hp = max_hp
        self.max_hp = max_hp

    def is_alive(self):
        """キャラクターが生存しているかを判定"""
        return self.hp > 0

    def reset_hp(self):
        """HPを最大値にリセット"""
        self.hp = self.max_hp

In [46]:
#@title 環境クラス

class GameEnvironment:
    def __init__(self):
        """
        バトル環境の初期化
        """
        self.character1 = None
        self.character2 = None
        self.turn = 1  # ターン数を記録

    def set_characters(self, char1, char2):
        """
        キャラクターをバトルにセット
        :param char1: Characterオブジェクト
        :param char2: Characterオブジェクト
        """
        self.character1 = char1
        self.character2 = char2

    def reset(self):
        """
        環境を初期状態にリセット
        """
        self.character1.reset_hp()
        self.character2.reset_hp()
        self.turn = 1

In [66]:
# @title 性格別報酬クラス

class RewardSystem:
    """
    性格ごとに行動を評価し、報酬を計算するクラス。
    """
    def __init__(self, personality):
        """
        コンストラクタ
        :param personality: 性格
        :param personality_weights: 性格ごとの行動別報酬
        """
        self.personality = personality
        self.personality_weights = {
          "気まぐれ": {"物理的攻撃": 10, "論理的攻撃": 10, "回復": 10, "防御": 10, "休憩": 10},
          "やんちゃ": {"物理的攻撃": 20, "論理的攻撃": 10, "回復": 0, "防御": 0, "休憩": 20},
          "引っ込み思案": {"物理的攻撃": -10, "論理的攻撃": 10, "回復": 20, "防御": 20, "休憩": 0},
          "優しい": {"物理的攻撃": -10, "論理的攻撃": -10, "回復": 10, "防御": 10, "休憩": 0},
        }

    def calculate_reward(self, action, last_action):
        """
        報酬を計算する
        :param action: 現在の行動
        :param last_action: 前回の行動
        :return: 報酬値
        """
        # 性格に応じた期待行動を参照
        base_reward = self.personality_weights[self.personality].get(action, 0)

         # ペナルティ
         # 改善の余地あり(性格によってはサボりはペナルティではなく報酬など)
        if action == "休憩":
            base_reward -= 10  # サボりのペナルティ
         # おそらく今はlast_actionを記録できていない
        if action == last_action:
            base_reward -= 40  # 反復行動のペナルティ

        return base_reward

In [48]:
# @title エージェントクラス

class Agent:
    """
    強化学習エージェントを定義するクラス。
    """
    def __init__(self, actions, learning_rate=0.1, discount_factor=0.9, epsilon=0.1):
        """
        コンストラクタ
        :param actions: アクションのリスト
        :param learning_rate: 学習率（α）
        :param discount_factor: 割引率（γ）
        :param epsilon: 探索率（ε）
        """
        self.actions = actions
        self.q_table = {}  # 状態-アクションペアの価値を格納する
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon
        self.action_count = {action: 0 for action in actions}  # 各アクションの選択回数を記録
        self.last_action = None  # 前回の行動

    def choose_action(self, state, exploration=True):
        """
        ε-greedyポリシーで行動を選択
        :param state: 現在の状態
        :return: 選択したアクション
        詳しいことわからない
        """
        if state not in self.q_table:
            self.q_table[state] = {action: 0 for action in self.actions}

        if exploration and random.random() < self.epsilon:
            # 探索（ランダム選択）
            action = random.choice(self.actions)
        else:
            # 活用（最大Q値の選択）
            action =  max(self.q_table[state], key=self.q_table[state].get)

        self.action_count[action] += 1
        self.last_action = action
        return action

    def learn(self, state, action, reward, next_state):
        """
        Q値を更新
        :param state: 現在の状態
        :param action: 実行したアクション
        :param reward: 環境から得られた報酬
        :param next_state: 次の状態
        """
        if next_state not in self.q_table:
            self.q_table[next_state] = {action: 0 for action in self.actions}

        # Q値更新式
        best_next_action = max(self.q_table[next_state], key=self.q_table[next_state].get)
        td_target = reward + self.discount_factor * self.q_table[next_state][best_next_action]
        td_error = td_target - self.q_table[state][action]
        self.q_table[state][action] += self.learning_rate * td_error

    def display_action_counts(self):
      """
      各アクションの選択回数を表示
      """
      print("=== 各アクションの選択回数 ===")
      for action, count in self.action_count.items():
        print(f"{action}: {count} 回")

In [64]:
# @title 学習ループと行動シミュレーション

def random_action():
    return random.choice(["物理的攻撃", "論理的攻撃", "回復", "防御", "休憩"])

def train_agent(environment, agent, episodes=100):
    """
    エージェントをトレーニングする関数
    :param environment: GameEnvironmentオブジェクト
    :param agent: Agentオブジェクト
    :param episodes: トレーニングエピソード数
    """
    for episode in range(episodes):
        # 環境をリセット
        environment.reset()
        state = (environment.character1.hp, environment.character2.hp)  # 状態: 各キャラのHP
        total_reward = 0
        agent.last_action = None

        while environment.character1.is_alive() and environment.character2.is_alive():
            # 行動を選択
            action1 = agent.choose_action(state)
            action2 = random_action()
            # 行動を環境に適用して次の状態と報酬を取得
            next_state, reward = simulate_action(environment, action1, action2, agent)
            agent.learn(state, action1, reward, next_state)  # 学習

            state = next_state  # 状態を更新
            total_reward += reward

        print(f"Episode {episode + 1}/{episodes}, Total Reward: {total_reward}")

def simulate_action(environment, action1, action2, agent):
    """
    両キャラクターの行動をシミュレーションし、次の状態と報酬を計算
    :param environment: GameEnvironmentオブジェクト
    :param action1: 先攻キャラクターの行動
    :param action2: 後攻キャラクターの行動
    :return: 次の状態, 報酬
    """
    char1 = environment.character1
    char2 = environment.character2
    reward_system = RewardSystem(char1.personality)

    # 先攻キャラクターの行動をシミュレート
    defense_effect = 0  # 防御の効果
    if action1 == "物理的攻撃":
        char2.hp = max(0, char2.hp - 30)
        if char2.hp == 0:
          # 報酬を計算
          reward = reward_system.calculate_reward(
            action=action1,
            last_action=agent.last_action  # 簡略化
          )
          return (char1.hp, char2.hp), reward
    elif action1 == "論理的攻撃":
        char2.hp = max(0, char2.hp - 30)
        if char2.hp == 0:
          # 報酬を計算
          reward = reward_system.calculate_reward(
            action=action1,
            last_action=agent.last_action  # 簡略化
          )
          return (char1.hp, char2.hp), reward
    elif action1 == "回復":
        char1.hp = min(char1.max_hp, char1.hp + 20)
    elif action1 == "防御":
        defense_effect = 10  # 次ターンのダメージを軽減
    elif action1 == "休憩":
        defense_effect = -10  # 次ターンのダメージを増加

    # 後攻キャラクターの行動をシミュレート
    if action2 == "物理的攻撃":
        char1.hp = max(0, char1.hp - 30 - defense_effect)  # 防御効果を適用
        if char1.hp == 0:
          # 報酬を計算
          reward = reward_system.calculate_reward(
            action=action1,
            last_action=agent.last_action  # 簡略化
          )
          return (char1.hp, char2.hp), reward - 40  # 死んだ場合は-40のペナルティ
    elif action2 == "論理的攻撃":
        char1.hp = max(0, char1.hp - 30 - defense_effect)  # 防御効果を適用
        if char1.hp == 0:
          # 報酬を計算
          reward = reward_system.calculate_reward(
            action=action1,
            last_action=agent.last_action  # 簡略化
          )
          return (char1.hp, char2.hp), reward - 40  # 死んだ場合は-40のペナルティ
    elif action2 == "回復":
        char2.hp = min(char2.max_hp, char2.hp + 20)
    elif action2 == "防御":
        pass  # 後攻キャラクターの防御効果はシミュレーションに影響しない
    elif action2 == "休憩":
        pass  # 後攻キャラクターの休憩はシミュレーションに影響しない

    # 報酬を計算
    reward = reward_system.calculate_reward(
        action=action1,
        last_action=agent.last_action  # 簡略化
    )

    return (char1.hp, char2.hp), reward

In [67]:
# @title 強化学習を実行

from google.colab import drive

def main():
    drive.mount('/content/drive') # Driveのマウント
    model_path = '/content/drive/My Drive/agent_model3.pkl' # 保存先のパスを指定

    # 環境の初期化
    environment = GameEnvironment()
    char1 = Character("Alice", "優しい")
    char2 = Character("Bob", "気まぐれ")
    environment.set_characters(char1, char2)

    # エージェントの初期化
    agent = Agent(
        actions=["物理的攻撃", "論理的攻撃", "回復", "防御", "休憩"]
    )

    # 学習エピソード数
    episodes = 10000

    # 学習
    print("=== 学習開始 ===")
    train_agent(environment, agent, episodes=episodes)
    print("=== 学習完了 ===")

    with open(model_path, 'wb') as file:
      pickle.dump(agent, file)
    print(f"モデルをGoogle Driveに保存: {model_path}")

    # 学習結果を表示
    agent.display_action_counts()

    # 学習結果を使用したエージェントの評価
    evaluate_agent(environment, agent, rounds=10)

def evaluate_agent(environment, agent, rounds=10):
    """
    学習後のエージェントを評価する関数
    :param environment: GameEnvironmentオブジェクト
    :param agent: 学習済みエージェント
    :param rounds: 評価のための試行回数
    """
    print("=== エージェント評価 ===")
    for round_num in range(1, rounds + 1):
        # 環境をリセット
        environment.reset()
        state = (environment.character1.hp, environment.character2.hp)

        total_reward = 0

        while environment.character1.is_alive() and environment.character2.is_alive():
            # エージェントの行動を選択
            action1 = agent.choose_action(state, exploration=False)  # 学習後なので探索をオフ
            # 後攻キャラクターの行動はランダム（またはルールベース）
            action2 = random_action()  # または rule_based_action(environment)

            # 環境に行動を適用して次の状態と報酬を取得
            next_state, reward = simulate_action(environment, action1, action2, agent)
            total_reward += reward

            # 状態を更新
            state = next_state

        print(f"Round {round_num}: Total Reward = {total_reward}")

if __name__ == "__main__":
    main()


ストリーミング出力は最後の 5000 行に切り捨てられました。
Episode 5020/10000, Total Reward: -250
Episode 5021/10000, Total Reward: -200
Episode 5022/10000, Total Reward: -200
Episode 5023/10000, Total Reward: -200
Episode 5024/10000, Total Reward: -200
Episode 5025/10000, Total Reward: -310
Episode 5026/10000, Total Reward: -200
Episode 5027/10000, Total Reward: -200
Episode 5028/10000, Total Reward: -200
Episode 5029/10000, Total Reward: -200
Episode 5030/10000, Total Reward: -250
Episode 5031/10000, Total Reward: -200
Episode 5032/10000, Total Reward: -350
Episode 5033/10000, Total Reward: -200
Episode 5034/10000, Total Reward: -200
Episode 5035/10000, Total Reward: -260
Episode 5036/10000, Total Reward: -200
Episode 5037/10000, Total Reward: -230
Episode 5038/10000, Total Reward: -300
Episode 5039/10000, Total Reward: -550
Episode 5040/10000, Total Reward: -200
Episode 5041/10000, Total Reward: -290
Episode 5042/10000, Total Reward: -200
Episode 5043/10000, Total Reward: -210
Episode 5044/10000, Total Reward